## Cleaning all messages from both users and contacts

### Method

- Read in the csv file
- extract just the emails to play with
- make a cleaning function
- check what the cleaning function is doing

In [2]:
from nltk import tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import pandas as pd
import sys

sys.path.insert(0, '../mining')
import connector
import query
from vader_sentiment_analysis import SentimentAnalyser
import os
from bs4 import BeautifulSoup
import re

/home/sophie/anaconda3/envs/py27/lib/python2.7/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [3]:
df = pd.read_csv("../data/SELECT_ALL_from_all_msgs_from_contacts_users.csv", 
                 header = 0)
df = df.drop(['Unnamed: 0'],axis=1)

/home/sophie/anaconda3/envs/py27/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
list(df)

['user_id', 'contact_id', 'relationship', 'channel', 'sent_at', 'message']

In [42]:
df_emails = df[df['channel'] == 'gmail']

In [43]:
emails = df_emails['message'].tolist()

In [51]:
df_emails['user_id'].unique()[0:10]

array([  10.,   35.,   47.,   59.,   81.,   96.,   97.,  114.,  116.,  144.])

In [54]:
df_emails[['user_id','contact_id','relationship','message']][df_emails['user_id'] == 35]

,user_id,contact_id,relationship,message
22447,35.0,54.0,Friend,"hey aldo,\nit was nice to meet you the other d..."
22448,35.0,54.0,Friend,"Hey Troy,\nIt's great to hear from you! I chec..."
22449,35.0,54.0,Friend,That sounds great! My only complication is a g...
22450,35.0,54.0,Friend,"Great! If you want, we can easily go on Saturd..."
22451,35.0,54.0,Friend,Alright. It looks like I'll be going to this T...
22452,35.0,54.0,Friend,"Yeah, let's do Saturday. Shall we meet at the ..."
22453,35.0,54.0,Friend,"Hey, hope the event went well yesterday. Could..."
22454,35.0,54.0,Friend,Hey Aldo! Sorry for not getting back to you.. ...
22455,35.0,54.0,Friend,No worries... I'll see you at the theater at 9...
22456,35.0,54.0,Friend,Hey Troy!\nIt was great seeing you the other d...


In [24]:
print df[['channel','message']][0:10]
print len(df)

  channel                                            message
0   gmail  hey aldo,\nit was nice to meet you the other d...
1   gmail  That sounds great! My only complication is a g...
2   gmail  Alright. It looks like I'll be going to this T...
3   gmail  Hey Aldo! Sorry for not getting back to you.. ...
4   gmail  Hey Aldo!\nSorry for the slow reply; I'd defin...
5   gmail  Hey Aldo! Thanks so much for thinking of me an...
6   gmail  Hey Aldo!\nNo problem re: disappearing. I've b...
7   gmail  you could've just e-mailed w/ an emoticon\n;)\...
8   gmail  hey aldo,\nhere is the link for the movie i wa...
9   gmail  feeling *mostly* alright :)\ndefinitely didn't...
2867448


In [35]:
sql_query = """SELECT DISTINCT contacts.user_id AS user_id,
feed_items.from_id AS contact_id, contact_types.name AS relationship, channels.name
AS channel, feed_items.send_at AS sent_at, feed_items.body AS message
FROM feed_items
JOIN channels
ON feed_items.channel_id = channels.id
JOIN contacts
ON feed_items.from_id = contacts.id
JOIN contact_types
ON contacts.contact_type_id = contact_types.id
JOIN users
ON contacts.user_id = users.id
WHERE contacts.is_fake = false
UNION
SELECT DISTINCT contacts.user_id AS user_id,
feed_items.from_id AS contact_id, contact_types.name AS relationship, 
channels.name
AS channel, feed_items.send_at AS sent_at, feed_items.body AS message
FROM feed_items
JOIN channels ON feed_items.channel_id = channels.id
JOIN feeds ON feeds.id = feed_items.feed_id
JOIN contacts ON contacts.user_id = feeds.contact_id
JOIN contact_types ON contacts.contact_type_id = contact_types.id
JOIN users ON contacts.user_id = users.id
WHERE contacts.is_fake = false
ORDER BY user_id, contact_id, sent_at"""

dbconnection = connector.ConnectDB()
conn = dbconnection.get_connection()
q = query.Query(conn, sql_query)
query_list = q.get_query_list()
query_df = q.get_query_dataframe()
#q.write_df_to_csv()
query_df.shape

(2763818, 6)

In [55]:
df_emails2 = query_df[query_df['channel'] == 'gmail']
df_emails2[['user_id','contact_id','relationship','message','sent_at']][df_emails2['user_id'] == 35]

,user_id,contact_id,relationship,message,sent_at
22445,35,-1,Friend,I'll make tabouli (w/ quinoa).\nFroese: are yo...,2011-04-04 04:28:25
22446,35,-1,Friend,oh man no fair i am leaving on the 7th\nOn 201...,2011-04-04 05:27:29
22447,35,-1,Friend,oh how i miss red envelopes...\ni'll be there ...,2011-04-05 05:23:02
22448,35,-1,Friend,haha very funny Ayanna. =),2011-04-05 12:32:07
22449,35,-1,Friend,"Yes me too, sorry.",2011-04-07 01:20:21
22450,35,-1,Friend,Yes. Thank you.,2011-04-07 21:57:05
22451,35,-1,Friend,Ayannas SURPRISE Graduation Party! \nDate: May...,2011-04-16 23:01:38
22452,35,-1,Friend,Hey all! So sorry but I need to change the da...,2011-04-17 11:05:16
22453,35,0,Friend,"Hey Troy,\nIt's great to hear from you! I chec...",2010-10-12 04:21:44
22454,35,0,Friend,"Great! If you want, we can easily go on Saturd...",2010-10-13 21:53:05


In [ ]:
df